In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from gensim.test.utils import datapath
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from collections import defaultdict
import sys
sys.path.append('../src')
from models import filter_terms_not_in_wemodel, \
    get_2ndorder_association_metric_list_for_target_list, \
    get_1storder_association_metric_list_for_target_list, \
    get_expSG_1storder_relation_no_cache_NEW, \
    get_expSG_1storder_relation_no_cache_NEW_ALLWORDS, \
    get_matrices_from_term_lists, \
    save_arrays, open_pickle, save_pickle, \
    save_scalers

In [4]:
we_model = KeyedVectors.load('../data/interim/glove_840_normed', mmap='r')
print('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')

loading done!
Total words: 2196016


C:\Users\danie\Miniconda3\envs\semproject2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
'''we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')
'''
'''
# Caliskan GloVe
glove_file = '../data/external/glove.6B/glove.6B.50d.txt'
_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')
we_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')
print('loading done!')
print(f'Total words: {len(we_model.wv.vocab)}')
'''

loading done!
Total words: 312425


"\n# Caliskan GloVe\nglove_file = '../data/external/glove.6B/glove.6B.50d.txt'\n_ = glove2word2vec(glove_file, '../data/interim/tmp.txt')\nwe_model = KeyedVectors.load_word2vec_format('../data/interim/tmp.txt')\nprint('loading done!')\nprint(f'Total words: {len(we_model.wv.vocab)}')\n"

In [3]:
FILEPATH = '../data/interim/association_metric_exps.pickle'
EXPERIMENT_DEFINITION_PATH = '../data/interim/experiment_definitions.pickle'
THRESHOLD_BIASES_PATH_2NDORDER = '../data/processed/threshold_biases_2ndorder.pickle'
THRESHOLD_BIASES_PATH_1STORDER = '../data/processed/threshold_biases_1storder.pickle'
SCALERS_FILEPATH = '../data/processed/scalers.pickle'

In [4]:
def add_experiment_definition(exp_num, X_terms, Y_terms, A_terms, B_terms, X_label, Y_label, A_label, B_label, filepath):
    dct = open_pickle(filepath)
    dct[exp_num]['X_terms'] = X_terms
    dct[exp_num]['Y_terms'] = Y_terms
    dct[exp_num]['A_terms'] = A_terms
    dct[exp_num]['B_terms'] = B_terms
    dct[exp_num]['X_label'] = X_label
    dct[exp_num]['Y_label'] = Y_label
    dct[exp_num]['A_label'] = A_label
    dct[exp_num]['B_label'] = B_label
    save_pickle(dct, filepath)

# Define the Experiments

In [5]:
# WEAT 1
exp_num = 1
X_label = 'Flowers'
Y_label = 'Insects'
A_label = 'Pleasant'
B_label = 'Unpleasant'
X_terms = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 
           'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil','lilac',
           'pansy','tulip','buttercup','daisy','lily','penny','violet','carnation', 'gladiola',
           'magnolia','petunia','zinnia']
Y_terms = ['ant','caterpillar','flea','locust','spider','bedbug','centipede','fly',
          'maggot','tarantula','bee','cockroach','gnat','mosquito','termite','beetle',
          'cricket','hornet','moth','wasp','blackfly','dragonfly','horsefly','roach',
          'weevil']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
X_terms, Y_terms = filter_terms_not_in_wemodel(we_model, X_terms, Y_terms)
A_terms, B_terms = filter_terms_not_in_wemodel(we_model, A_terms, B_terms)
add_experiment_definition(exp_num, X_terms, Y_terms, A_terms, B_terms, 
                          X_label, Y_label, A_label, B_label, EXPERIMENT_DEFINITION_PATH)

The following terms were removed from the list first_list because they were not found in the we_model: ['gladiola']
The following terms were removed from the list second_list because they were not found in the we_model: []
The following terms were removed from the second list to balance the length of the lists: ['ant']
The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []


In [12]:
# WEAT 2
exp_num = 2
X_label = 'Instruments'
Y_label = 'Weapons'
A_label = 'Pleasant'
B_label = 'Unpleasant'
X_terms = ['bagpipe','cello','guitar','lute','trombone','banjo','clarinet','harmonica',
           'mandolin','trumpet','bassoon','drum','harp','oboe','tuba','bell','fiddle',
           'harpsichord','piano','viola','bongo','flute','horn','saxophone']
Y_terms = ['arrow','club','gun','missile','spear','axe','dagger','harpoon','pistol',
          'sword','blade','dynamite','hatchet','rifle','tank','bomb','firearm',
          'knife','shotgun','teargas','cannon','grenade','mace','slingshot','whip']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
X_terms, Y_terms = filter_terms_not_in_wemodel(we_model, X_terms, Y_terms)
A_terms, B_terms = filter_terms_not_in_wemodel(we_model, A_terms, B_terms)
add_experiment_definition(exp_num, X_terms, Y_terms, A_terms, B_terms, 
                          X_label, Y_label, A_label, B_label, EXPERIMENT_DEFINITION_PATH)

The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []
The following terms were removed from the second list to balance the length of the lists: ['arrow']
The following terms were removed from the list first_list because they were not found in the we_model: []
The following terms were removed from the list second_list because they were not found in the we_model: []


# Second-Order

In [11]:
def calculate_cosines_for_target_word_unscaled(word_vec, A_mtx, B_mtx):
    A_dot_v = np.dot(A_mtx, word_vec)
    B_dot_v = np.dot(B_mtx, word_vec)
    A_norms = np.multiply(np.linalg.norm(A_mtx, axis=1), np.linalg.norm(word_vec))
    B_norms = np.multiply(np.linalg.norm(B_mtx, axis=1), np.linalg.norm(word_vec))
    A_cosines = np.divide(A_dot_v, A_norms)
    B_cosines = np.divide(B_dot_v, B_norms)
    return np.mean(A_cosines), np.mean(B_cosines)

def calculate_cosines_for_all_words_unscaled(we_model, A_mtx, B_mtx):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    #A_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, A_mtx)
    #B_cosines_apply = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, B_mtx)
    A_mtx_norm = A_mtx/np.linalg.norm(A_mtx, axis=1).reshape(-1,1)
    B_mtx_norm = B_mtx/np.linalg.norm(B_mtx, axis=1).reshape(-1,1)
    all_mtx_norm = we_model.wv.vectors/np.linalg.norm(we_model.wv.vectors, axis=1).reshape(-1,1)
    
    all_associations_to_A = np.dot(A_mtx_norm, all_mtx_norm.T)
    all_associations_to_B = np.dot(B_mtx_norm, all_mtx_norm.T)
    
    return np.mean(all_associations_to_A, axis=0), np.mean(all_associations_to_B, axis=0)


def get_2ndorder_association_metric_list_for_target_list(target_list, A_terms, B_terms, we_model, exp_num):
    
    [X_mtx, _, A_mtx, B_mtx] = get_matrices_from_term_lists(we_model, target_list, target_list, A_terms, B_terms)
    
    # A_associations, B_associations are associations for all words    
    A_associations, B_associations = calculate_cosines_for_all_words_unscaled(we_model, A_mtx, B_mtx)
    
    
    all_associations = np.concatenate((A_associations, B_associations))
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit(all_associations.reshape(-1,1))
    save_scalers(SCALERS_FILEPATH, exp_num, 'second', scaler)
    
    _th = np.mean(np.abs(A_associations - B_associations))
    _th = scaler.transform(_th.reshape(-1, 1))[0,0]
               
    threshold_biases = open_pickle(THRESHOLD_BIASES_PATH_2NDORDER)
    threshold_biases = scaler.transform(threshold_biases.reshape(-1,1))
    pct_5 = np.percentile(threshold_biases, 5)
    pct_95 = np.percentile(threshold_biases, 95)
    
    biases = A_associations - B_associations
    biases = scaler.transform(biases.reshape(-1, 1))
    lower_bound = np.percentile(biases, 5)
    print(f'Lower bound: {lower_bound}')
    upper_bound = np.percentile(biases, 95)
    print(f'Upper bound: {upper_bound}')
    
    target_associations = np.apply_along_axis(lambda x_vec: calculate_cosines_for_target_word_unscaled(x_vec, A_mtx, B_mtx), 1, X_mtx)
    
    target_biases = []
    A_biases = []
    for _assoc in target_associations:
        _A_assoc = scaler.transform(_assoc[0].reshape(-1, 1))[0,0]
        _B_assoc = scaler.transform(_assoc[1].reshape(-1, 1))[0,0]
        _bias = _A_assoc - _B_assoc
        target_biases.append(_bias)
        A_biases.append(_A_assoc)
    return np.array(target_biases), _th, pct_5, pct_95, np.array(A_biases), lower_bound, upper_bound

def run_exps_2ndorder(X_terms, Y_terms, A_terms, B_terms, exp_num):
    order='second'
    X_metrics, _th, pct_5, pct_95, A_biases, lower_bound, upper_bound = get_2ndorder_association_metric_list_for_target_list(X_terms, A_terms, B_terms, we_model, exp_num)
    Y_metrics, _th, pct_5, pct_95, A_biases, lower_bound, upper_bound = get_2ndorder_association_metric_list_for_target_list(Y_terms, A_terms, B_terms, we_model, exp_num)
    print (X_metrics)
    print (Y_metrics)

    print ('mean bias to X', np.mean(X_metrics))
    print ('mean bias to Y', np.mean(Y_metrics))

    print ('Bias threshold', _th)
    print ('5th percentile', pct_5)
    print ('95th percentile', pct_95)

    order = 'second'
    threshold = _th
    save_arrays(FILEPATH, exp_num, order, X_metrics, Y_metrics, threshold, pct_5, pct_95, A_biases, lower_bound, upper_bound)
run_exps_2ndorder(X_terms, Y_terms, A_terms, B_terms, exp_num)

Lower bound: -0.12051583379507065
Upper bound: 0.10539177060127258
Lower bound: -0.12051583379507065
Upper bound: 0.10539177060127258
[ 0.06463376  0.13652503 -0.04676503  0.11253601  0.04916599  0.17652243
  0.09883526  0.04855525  0.07594025  0.16368937  0.08388424  0.11431959
  0.13489187  0.08018041  0.1137642   0.09362984  0.10222384  0.09800541
  0.14387575  0.04530728  0.09423137  0.14132568  0.06971878  0.06853676]
[-0.02787289 -0.01819867 -0.06967109 -0.01150721 -0.1667692  -0.06400925
  0.04445949 -0.18263587 -0.07073748  0.11490756 -0.09649387 -0.04441088
 -0.11249268 -0.08999455 -0.10317296  0.02992854  0.0309861  -0.02143282
 -0.06515384  0.02837816  0.04654327  0.03567371  0.03692234 -0.11014193]
mean bias to X 0.09431389
mean bias to Y -0.036954004
Bias threshold 0.0380725
5th percentile 0.006891302671283487
95th percentile 0.557002368569374
Results array successfully saved to file ../data/interim/association_metric_exps.pickle under keys [1][second]


In [13]:
def run_all_exps(order='second'):
    exps = open_pickle(EXPERIMENT_DEFINITION_PATH)
    print(f'ORDER = {order}')
    for exp_num, exp in exps.items():
        print('***********************************')
        print(f'Experiment: {exp_num}')
        X_terms = exp['X_terms']
        Y_terms = exp['Y_terms']
        A_terms = exp['A_terms']
        B_terms = exp['B_terms']
        if order == 'second':
            run_exps_2ndorder(X_terms, Y_terms, A_terms, B_terms, exp_num)
        else:
            run_exps_1storder(X_terms, Y_terms, A_terms, B_terms, exp_num)
run_all_exps(order='second')

ORDER = second
***********************************
Experiment: 1
Lower bound: -0.12051583379507065
Upper bound: 0.10539177060127258
Lower bound: -0.12051583379507065
Upper bound: 0.10539177060127258
[ 0.06463376  0.13652503 -0.04676503  0.11253601  0.04916599  0.17652243
  0.09883526  0.04855525  0.07594025  0.16368937  0.08388424  0.11431959
  0.13489187  0.08018041  0.1137642   0.09362984  0.10222384  0.09800541
  0.14387575  0.04530728  0.09423137  0.14132568  0.06971878  0.06853676]
[-0.02787289 -0.01819867 -0.06967109 -0.01150721 -0.1667692  -0.06400925
  0.04445949 -0.18263587 -0.07073748  0.11490756 -0.09649387 -0.04441088
 -0.11249268 -0.08999455 -0.10317296  0.02992854  0.0309861  -0.02143282
 -0.06515384  0.02837816  0.04654327  0.03567371  0.03692234 -0.11014193]
mean bias to X 0.09431389
mean bias to Y -0.036954004
Bias threshold 0.0380725
5th percentile 0.006891302671283487
95th percentile 0.557002368569374
Results array successfully saved to file ../data/interim/associati